<a href="https://colab.research.google.com/github/aakhterov/ML_projects/blob/master/Text_classification/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# !pip install pymorphy2

In [9]:
import json
import re
from pprint import pprint
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

#Load data

In [10]:
# Load conversations

In [11]:
with open("/content/drive/MyDrive/Colab Notebooks/Data/samples.json", mode="r") as file:
    dialogs_json = json.loads(file.read())

In [12]:
dialogs_json["50980"]

{'er': 245952,
 'site': 159859,
 'created_at': '2020-04-30 18:38:10',
 'visitors_phrases': ['Здравствуйте!',
  'Подскажите как добавить устройство к договору своему',
  'С которого с Вами общаюсь да',
  'Ноутбук возможно',
  'Захожу с него в кабинет договора 111111',
  'A1-B1-C1-D4-E4-F7',
  'Спасибо',
  '1 мб/с :('],
 'agents_phrases': ['Здравствуйте!',
  'Данное устройство уже было ранее зарегистрировано в сети ловит?',
  'Ноутбук был ране у нас на другом договоре зарегистрирован? Если да то уточните номер договора.',
  'Сергей уточните',
  'Сейчас переведу устройство на действующий договор',
  'Сергей',
  'Скорость у городской сети до 10мбит в сек',
  'Возможно',
  'Спасибо']}

In [13]:
# Load conversation themes

In [14]:
er = {}
with open("/content/drive/MyDrive/Colab Notebooks/Data/dialogs_table_employee_remarks.sql", mode="r") as file:
    for line in file:
        if line[0] == '(':
            rec = line[1:-3].split(',')
            er[int(rec[0])] = rec[1].strip(" '").replace("\\","").replace("\"","")
pprint(er)

{1: 'нет оценки',
 236486: 'Заявка на подключение Интернет',
 236487: 'Консультация по подключению Интернет',
 236488: 'Консультация по КТВ',
 236489: 'Финансовая консультация',
 236490: 'Консультация по интерактивному телевидению',
 236491: 'Не работает ЛК приложение – рекомендация по работе',
 245949: 'ТТ Финансы',
 245950: 'ТТ Клиентские',
 245951: 'Общая проблема - информирование клиента',
 245952: 'Не работает интернет – рекомендация по настройке',
 245953: 'Смена тарифного плана',
 245954: 'Инсталляция',
 245955: 'Консультация по доп.услугам',
 245956: 'Прочее',
 246249: 'ТТ Телевидение',
 246250: 'Не работает КТВ – рекомендация по настройке',
 246251: 'Не работает Интерактивное ТВ – рекомендация по настройке',
 246252: 'Не работают прочие услуги – рекомендация по работе',
 247714: 'Возврат ДС/оборудования // Переоформление /расторжение договора'}


In [15]:
# Grouping conversation topics to group related topics

In [16]:
categories_to_er = {
    "connect_to_inet": [236486, 236487],
    "install": [245954],
    "finance": [236489, 245949],
    "no_LK": [236491],
    "no_ITV": [246251],
    "no_KTV": [246250],
    "low_speed": [245950],
    "no_internet": [245952],
    "termination_return": [247714],
    "tariff_changes": [245953],
    "TV": [246249],
    "KTV": [236488],
    "ITV": [236490],
    "additional": [245955],
#     "other": [245953, 246249, 236488, 236490, 245955]
}
er_to_categories = {}
for category, value in categories_to_er.items():
    er_to_categories = {**er_to_categories, **{er: category for er in value}}
pprint(er_to_categories)

{236486: 'connect_to_inet',
 236487: 'connect_to_inet',
 236488: 'KTV',
 236489: 'finance',
 236490: 'ITV',
 236491: 'no_LK',
 245949: 'finance',
 245950: 'low_speed',
 245952: 'no_internet',
 245953: 'tariff_changes',
 245954: 'install',
 245955: 'additional',
 246249: 'TV',
 246250: 'no_KTV',
 246251: 'no_ITV',
 247714: 'termination_return'}


#Preprocessing pipeline

In [17]:
# Bring the words associated with wi-fi to the same form

In [18]:
wifi_union = [("wi", "fi"), ("вай", "фай")]
wifi_naming = {"wifi": "вайфай", "wi-fi": "вайфай", "вайфая": "вайфай"}
def wifi_preprocessing(phrase: str) -> str:
    words = [word.replace("фая", "фай") for word in phrase.split()]
    i = 0
    united_words = []
    while i<len(words):
        for pair in wifi_union:
            if i < len(words)-1 and words[i] == pair[0] and words[i+1] == pair[1]:
                united_words.append(words[i] + words[i+1])
                i += 2
                break
        else:
            united_words.append(words[i])
            i += 1
    words = [word if word not in wifi_naming else wifi_naming[word] for word in united_words]
    result_phrase = " ".join(words)
    return result_phrase

In [19]:
# Remove links

In [20]:
def remove_links(text: str) -> str:
    return re.sub(r'(www|http:|https:)+[^\s]+[\w]', '', text)

In [21]:
# Remove all expected digits, russian letters, space, ., ! and ?

In [22]:
def remove_punct_and_latin(text: str) -> str:
    return re.sub(r'[^0-9А-ЯЁа-яё\s.!?]', '', text)

In [23]:
# Replace digits by symbol #

In [24]:
def replace_digits(text: str) -> str:
    for length in range(10,0,-1):
        pattern = r'\d{' + str(length) + '}'
        match = re.search(pattern, text)
        if match:
            text = re.sub(pattern, '#'*length, text)
    return text

In [25]:
# Converting words to normal form

In [26]:
exception_words = ['ловит', 'ловита', 'ловиту']
def normalize(text: str) -> str:
    normilized_sentence = []
    for word in text.split():
        if word not in exception_words:
            word_forms = morph.parse(word)
            if word_forms:
                normilized_sentence.append(word_forms[0].normal_form)
        else:
            normilized_sentence.append(word)
    return " ".join(normilized_sentence)

In [27]:
# Split phrase by sentences

In [28]:
def split_by_sentence(separators, phrases):
    tmp = phrases
    for sep in separators:
        result = []
        for phrase in list(map(lambda x: x.split(sep), tmp)):
            result += phrase
        tmp = result
    return [phrase.strip() for phrase in result if phrase.strip() != ""]

In [29]:
# Remove stop words

In [30]:
def remove_stop_words(text: str) -> str:
    stop_words = ['здравствуйте', 'добрый день', 'доброе утро', 'доброй ночи', 'добрый вечер', 'привет', 'спасибо',
                  'добрый утро', 'добрый ночи', 'досвидание', 'до свидание', 'всего хорошего', 'всего хороший']
    for word in stop_words:
        pattern = r'' + word + ''
        text = re.sub(pattern, '', text)
    return text

In [31]:
# Make transformation

In [32]:
all_processing_phrases = []
for _, value in dialogs_json.items():
    phrases = value["visitors_phrases"] + value["agents_phrases"]
    phrases = list(map(lambda x: remove_links(x), phrases))
    phrases = list(map(lambda x: wifi_preprocessing(x), phrases))
    phrases = list(map(lambda x: remove_punct_and_latin(x), phrases))
    phrases = list(map(lambda x: replace_digits(x), phrases))
    phrases = list(map(lambda x: x.lower(), phrases))
    phrases = split_by_sentence(".!?", phrases)
    phrases = list(map(lambda x: normalize(x), phrases))
    phrases = list(map(lambda x: remove_stop_words(x), phrases))
    phrases = [phrase.strip() for phrase in phrases if phrase.strip() !=""]
    all_processing_phrases += phrases

In [33]:
len(all_processing_phrases)

52

In [34]:
all_processing_phrases[:10]

['подсказать как добавить устройство к договор свой',
 'с который с вы общаться да',
 'ноутбук возможно',
 'заходить с он в кабинет договор ######',
 '######',
 '# мбс',
 'данный устройство уже быть ранее зарегистрировать в сеть ловит',
 'ноутбук быть рана у мы на друг договор зарегистрировать',
 'если да то уточнить номер договор',
 'сергей уточнить']

#Look at the length of the phrases

In [35]:
from statistics import mean, quantiles
import numpy as np

In [49]:
# Get lengths

In [54]:
lens = [len(phrase.split()) for phrase in all_processing_phrases]

In [55]:
mean(lens)

4.230769230769231

In [56]:
quantiles(lens)

[2.0, 4.0, 6.0]

In [57]:
np.bincount(lens) # get distribution by lengths

array([ 0,  9, 10,  3, 11,  2,  6,  4,  2,  3,  1,  1])

In [59]:
# Let's look at the phrases with length = 1

In [58]:
[phrase for phrase in all_processing_phrases if len(phrase.split()) == 1]

['######',
 'сергей',
 'возможно',
 'да',
 'да',
 'понятно',
 '####',
 '######',
 'уточнить']

In [51]:
# These phrases are meaningless. We can remove them

#Remove phrase with length 1

In [52]:
processing_phrases_more_one_word = [phrase for phrase in all_processing_phrases if len(phrase.split()) > 1]

In [53]:
len(processing_phrases_more_one_word) # there were 749130 phrases in reality

43